## Overview

Question: What datasets will provide the best performing model as measured by F1 accuracy?

### 1. Experiment 1: GPT3 by itself

Let's start first with using GPT3.5 zeroshot data only. For simplicity, we'll allow `prodigy train` to split for the evaluation dataset.

In [2]:
!python -m prodigy db-in hmwk_1_zeroshot data/gpt3-5-zeroshot.jsonl

✔ Created unstructured dataset 'hmwk_1_zeroshot' in database SQLite
✔ Imported 500 annotated examples and saved them to 'hmwk_1_zeroshot'
(session 2024-02-14_11-05-16) in database SQLite
Found and keeping existing "answer" in 0 examples


In [3]:
!python -m prodigy train --ner hmwk_1_zeroshot ./output/experiment-1 --label-stats --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Initializing pipeline ===========================
[2024-02-14 11:05:18,442] [INFO] Set up nlp object from config
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 400 | Evaluation: 100 (20% split)
Training: 400 | Evaluation: 100
Labels: ner (3)
[2024-02-14 11:05:18,533] [INFO] Pipeline: ['tok2vec', 'ner']
[2024-02-14 11:05:18,535] [INFO] Created vocabulary
[2024-02-14 11:05:18,535] [INFO] Finished initializing nlp object
[2024-02-14 11:05:19,315] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 400 | Evaluation: 100 (20% split)
Training: 400 | Evaluation: 100
Labels: 

In [4]:
!python -m prodigy train-curve --ner hmwk_1_zeroshot --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Train curve diagnostic ===========================
Training 4 times with 25%, 50%, 75%, 100% of the data

%      Score    ner   
----   ------   ------
  0%   0.00     0.00  
 25%   0.24 ▲   0.24 ▲
 50%   0.29 ▲   0.29 ▲
 75%   0.30 ▲   0.30 ▲
100%   0.34 ▲   0.34 ▲

✔ Accuracy improved in the last sample
As a rule of thumb, if accuracy increases in the last segment, this could
indicate that collecting more annotations of the same type will improve the
model further.


This gives us evidence that we can do better. But before adding more training data, let's add a dedicated hold out (evaluation) dataset. This is to allow us to have a consistent evaluation dataset across all of our subsequent experiments.

### 2. Experiment 2: GPT3 with holdout

Let's now load an evaluation dataset (200 unlabeled), retrain on `hmwk-1-zeroshot` and eval on `hmwk-1-eval`.

For the evaluation dataset, I used a different data 200 data. So in your case you may use `ner.manual` or `ner.correct`, but in my case I only needed to load the data like:

In [5]:
!python -m prodigy db-in hmwk_1_eval data/eval-reddit.jsonl

✔ Created unstructured dataset 'hmwk_1_eval' in database SQLite
✔ Imported 200 annotated examples and saved them to 'hmwk_1_eval'
(session 2024-02-14_11-10-37) in database SQLite
Found and keeping existing "answer" in 200 examples


In [6]:
!python -m prodigy train --ner hmwk_1_zeroshot,eval:hmwk_1_eval ./output/experiment-2  --label-stats --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Initializing pipeline ===========================
[2024-02-14 11:10:39,029] [INFO] Set up nlp object from config
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 500 | Evaluation: 200 (from datasets)
Training: 500 | Evaluation: 200
Labels: ner (3)
[2024-02-14 11:10:39,139] [INFO] Pipeline: ['tok2vec', 'ner']
[2024-02-14 11:10:39,141] [INFO] Created vocabulary
[2024-02-14 11:10:39,141] [INFO] Finished initializing nlp object
[2024-02-14 11:10:40,001] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 500 | Evaluation: 200 (from datasets)
Training: 500 | Evaluation: 200


Interestingly, the **F1 score** went up to 0.41 from 0.36. We need to be cautious before assigning causation (e.g., was it the increase in training size from 400 to 500? was it differences in the evaluation dataset?). It's not clear, but for the rest of the experiments we'll keep the 0.41 as our benchmark.

It's also worth noting that `INGREDIENT` has a much higher accuracy than `EQUIPMENT` and especially `DISH`. Any ideas on why that could be? What does it mean when Precision (P) is much higher than Recall (R)?

In [7]:
!python -m prodigy train-curve --ner hmwk_1_zeroshot,eval:hmwk_1_eval --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Train curve diagnostic ===========================
Training 4 times with 25%, 50%, 75%, 100% of the data

%      Score    ner   
----   ------   ------
  0%   0.00     0.00  
 25%   0.32 ▲   0.32 ▲
 50%   0.34 ▲   0.34 ▲
 75%   0.34     0.34  
100%   0.41 ▲   0.41 ▲

✔ Accuracy improved in the last sample
As a rule of thumb, if accuracy increases in the last segment, this could
indicate that collecting more annotations of the same type will improve the
model further.


We can also run `train-curve` to see if we've "leveled off", i.e., what's the extra value from adding more training data.

It seems that we should consider adding more training data. But before we do - let's train with only the Workshop data.


### 3. Experiment 3: Workshop with holdout

Let's now train only with the workshop data, but keep the same evaluation dataset.

In [8]:
!python -m prodigy db-in hmwk_1_workshop data/pydata-nyc-2023.jsonl

✔ Created unstructured dataset 'hmwk_1_workshop' in database SQLite
✔ Imported 1250 annotated examples and saved them to 'hmwk_1_workshop'
(session 2024-02-14_11-16-56) in database SQLite
Found and keeping existing "answer" in 1250 examples


Train with only the workshop data.

In [9]:
!python -m prodigy train --ner hmwk_1_workshop,eval:hmwk_1_eval ./output/experiment-3 --label-stats --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Initializing pipeline ===========================
[2024-02-14 11:16:58,517] [INFO] Set up nlp object from config
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 1183 | Evaluation: 200 (from datasets)
Training: 1163 | Evaluation: 200
Labels: ner (3)
[2024-02-14 11:16:58,727] [INFO] Pipeline: ['tok2vec', 'ner']
[2024-02-14 11:16:58,729] [INFO] Created vocabulary
[2024-02-14 11:16:58,729] [INFO] Finished initializing nlp object
[2024-02-14 11:17:00,293] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 1183 | Evaluation: 200 (from datasets)
Training: 1163 | Evaluation: 

We find that the workshop data as the training evaluates much better (**0.62** F1 score) on my evaluation dataset. Therefore, it's safe to say that the Workshop data can better predict my annotations than the GPT3 labeled trained model.

That's a big jump so there seems to be a lot of value in the Workshop Data. But it's also important to note that the Workshop dataset is almost 2x larger than the GPT3.5 dataset, so it's not clear: is it quantity or quality?

Notice that `INGREDIENT` accuracy, especially recall, improved a lot as did `EQUIPMENT` and `DISH`.

Let's now run `train-curve` to see if there's evidence that we may improve model performance if we label some more data.

In [17]:
!python -m prodigy train-curve --ner hmwk_1_workshop,eval:hmwk_1_eval --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Train curve diagnostic ===========================
Training 4 times with 25%, 50%, 75%, 100% of the data

%      Score    ner   
----   ------   ------
  0%   0.01     0.01  
 25%   0.48 ▲   0.48 ▲
 50%   0.55 ▲   0.55 ▲
 75%   0.61 ▲   0.61 ▲
100%   0.62 ▲   0.62 ▲

✔ Accuracy improved in the last sample
As a rule of thumb, if accuracy increases in the last segment, this could
indicate that collecting more annotations of the same type will improve the
model further.


### 4. Experiment 4: Workshop & GPT as training

Let's now experiment with adding the GPT3 Data, combining it to the training dataset.

In [10]:
!python -m prodigy db-merge hmwk_1_zeroshot,hmwk_1_workshop hmwk_1_train_exp4 

✔ Created dataset 'hmwk_1_train_exp4'
✔ Merged 1750 examples from 2 datasets
Created merged dataset 'hmwk_1_train_exp4'


In [11]:
!python -m prodigy train --ner hmwk_1_train_exp4,eval:hmwk_1_eval ./output/experiment-4 --label-stats --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Initializing pipeline ===========================
[2024-02-14 11:20:21,280] [INFO] Set up nlp object from config
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 1683 | Evaluation: 200 (from datasets)
Training: 1577 | Evaluation: 200
Labels: ner (3)
[2024-02-14 11:20:21,558] [INFO] Pipeline: ['tok2vec', 'ner']
[2024-02-14 11:20:21,560] [INFO] Created vocabulary
[2024-02-14 11:20:21,560] [INFO] Finished initializing nlp object
[2024-02-14 11:20:23,584] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 1683 | Evaluation: 200 (from datasets)
Training: 1577 | Evaluation: 

Interestingly, we found that actually including the GPT3 annotated dataset **decreases** accuracy slightly. We need to be careful about inferring significance, but this provides evidence that we may be better of not using the GPT3 data at all!

### 5. Experiment 5: Correct GPT3

To test this idea, let's now examine some of the GPT3.5 data. By reviewing we can not only correct some data, we can possibly develop insight to when the model is performing inconsistently. But to do so, it would be helpful to see the GPT3.5 labels **and** our best model (experiment 2) predictions at the same time.

Luckily, there's a `model-as-annotator` Prodigy recipe that allows you to do this.

I've commented out below because I previously saved this step as `data/hmwk-1-review.jsonl`. This allows me to fully rerun the code.

In [19]:
!prodigy ner.model-annotate hmwk_1_workshop_model ./output/experiment-3/model-best data/gpt3-5-zeroshot.jsonl ner_v3

Getting labels from the 'ner' component
Using 3 labels: ['DISH', 'EQUIPMENT', 'INGREDIENT']
100%|████████████████████████████████████████| 500/500 [00:01<00:00, 376.48it/s]


In [21]:
!python -m prodigy review hmwk_1_review hmwk_1_workshop_model,hmwk_1_zeroshot --label DISH,EQUIPMENT,INGREDIENT --auto-accept


Using 3 label(s): DISH, EQUIPMENT, INGREDIENT
Added dataset hmwk_1_review to database SQLite.

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

^C

✔ Saved 100 annotations to database SQLite
Dataset: hmwk_1_review
Session ID: 2024-02-14_14-30-25



In [22]:
!python -m prodigy db-out hmwk_1_review > data/hmwk-1-review.jsonl 

In [30]:
!wc -l data/hmwk-1-review.jsonl

     177 data/hmwk-1-review.jsonl


Notice that while we saved 100 examples, we have 77. That's because we choose `--auto-accept` so we also automatically kept any examples where our model and GPT3 annotation agreed.

In [32]:
!python -m prodigy db-merge hmwk_1_workshop,hmwk_1_review hmwk_1_train_exp5 

✔ Created dataset 'hmwk_1_train_exp5'
✔ Merged 1427 examples from 2 datasets
Created merged dataset 'hmwk_1_train_exp5'


In [33]:
!python -m prodigy train --ner hmwk_1_train_exp5,eval:hmwk_1_eval ./output/experiment-5 --label-stats --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Initializing pipeline ===========================
[2024-02-14 14:51:25,874] [INFO] Set up nlp object from config
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 1360 | Evaluation: 200 (from datasets)
Training: 1263 | Evaluation: 200
Labels: ner (3)
[2024-02-14 14:51:26,130] [INFO] Pipeline: ['tok2vec', 'ner']
[2024-02-14 14:51:26,132] [INFO] Created vocabulary
[2024-02-14 14:51:26,133] [INFO] Finished initializing nlp object
[2024-02-14 14:51:27,826] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 1360 | Evaluation: 200 (from datasets)
Training: 1263 | Evaluation: 

In [34]:
!python -m prodigy train-curve --ner hmwk_1_train_exp5,eval:hmwk_1_eval --training.patience=800

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
✔ Generated training config

=========================== Train curve diagnostic ===========================
Training 4 times with 25%, 50%, 75%, 100% of the data

%      Score    ner   
----   ------   ------
  0%   0.00     0.00  
 25%   0.48 ▲   0.48 ▲
 50%   0.58 ▲   0.58 ▲
 75%   0.60 ▲   0.60 ▲
100%   0.62 ▲   0.62 ▲

✔ Accuracy improved in the last sample
As a rule of thumb, if accuracy increases in the last segment, this could
indicate that collecting more annotations of the same type will improve the
model further.


### 6. Modify Word Vectors

We're going to follow [Prodigy docs](https://prodi.gy/docs/named-entity-recognition#transfer-learning) to improve the model by using word vectors (embeddings). 

In [36]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 4.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [37]:
!python -m prodigy train --ner hmwk_1_workshop,eval:hmwk_1_eval ./output/experiment-6 --base-model en_core_web_lg --label-stats

ℹ Using CPU

========================= Generating Prodigy config =========================
ℹ Auto-generating config with spaCy
ℹ Using config from base model
✔ Generated training config

=========================== Initializing pipeline ===========================
[2024-02-14 15:14:46,481] [INFO] Set up nlp object from config
Components: ner
Merging training and evaluation data for 1 components
  - [ner] Training: 1183 | Evaluation: 200 (from datasets)
Training: 1163 | Evaluation: 200
Labels: ner (3)
[2024-02-14 15:14:46,894] [INFO] Pipeline: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
[2024-02-14 15:14:46,894] [INFO] Resuming training for: ['ner', 'tok2vec']
[2024-02-14 15:14:46,902] [INFO] Created vocabulary
[2024-02-14 15:14:48,420] [INFO] Added vectors: en_core_web_lg
[2024-02-14 15:14:49,431] [INFO] Finished initializing nlp object
[2024-02-14 15:14:49,431] [INFO] Initialized pipeline components: []
✔ Initialized pipeline

============================= 

### Save models as wheel files

As an alternative to zip files, I'll save the models as wheel files.

In [38]:
!mkdir -p packages
!python -m spacy package ./output/experiment-1/model-best ./packages --name ner_reddit_cooking --version 1.0.0 --build wheel
!python -m spacy package ./output/experiment-2/model-best ./packages --name ner_reddit_cooking --version 2.0.0 --build wheel
!python -m spacy package ./output/experiment-3/model-best ./packages --name ner_reddit_cooking --version 3.0.0 --build wheel
!python -m spacy package ./output/experiment-4/model-best ./packages --name ner_reddit_cooking --version 4.0.0 --build wheel
!python -m spacy package ./output/experiment-5/model-best ./packages --name ner_reddit_cooking --version 5.0.0 --build wheel
!python -m spacy package ./output/experiment-6/model-best ./packages --name ner_reddit_cooking --version 6.0.0 --build wheel

ℹ Building package artifacts: wheel
✔ Loaded meta.json from file
output/experiment-1/model-best/meta.json
✔ Generated README.md from meta.json
✔ Successfully created package directory
'en_ner_reddit_cooking-1.0.0'
packages/en_ner_reddit_cooking-1.0.0
running bdist_wheel
running build
running build_py
creating build
creating build/lib
creating build/lib/en_ner_reddit_cooking
copying en_ner_reddit_cooking/__init__.py -> build/lib/en_ner_reddit_cooking
creating build/lib/en_ner_reddit_cooking/en_ner_reddit_cooking-1.0.0
copying en_ner_reddit_cooking/en_ner_reddit_cooking-1.0.0/tokenizer -> build/lib/en_ner_reddit_cooking/en_ner_reddit_cooking-1.0.0
copying en_ner_reddit_cooking/en_ner_reddit_cooking-1.0.0/config.cfg -> build/lib/en_ner_reddit_cooking/en_ner_reddit_cooking-1.0.0
copying en_ner_reddit_cooking/en_ner_reddit_cooking-1.0.0/README.md -> build/lib/en_ner_reddit_cooking/en_ner_reddit_cooking-1.0.0
copying en_ner_reddit_cooking/en_ner_reddit_cooking-1.0.0/meta.json -> build/lib/en

Let's now copy `cp` the files to the `models` folder.

In [39]:
!mkdir -p models
!cp -n packages/en_ner_reddit_cooking-1.0.0/dist/en_ner_reddit_cooking-1.0.0-py3-none-any.whl models/en_ner_reddit_cooking-1.0.0-py3-none-any.whl
!cp -n packages/en_ner_reddit_cooking-2.0.0/dist/en_ner_reddit_cooking-2.0.0-py3-none-any.whl models/en_ner_reddit_cooking-2.0.0-py3-none-any.whl
!cp -n packages/en_ner_reddit_cooking-3.0.0/dist/en_ner_reddit_cooking-3.0.0-py3-none-any.whl models/en_ner_reddit_cooking-3.0.0-py3-none-any.whl
!cp -n packages/en_ner_reddit_cooking-4.0.0/dist/en_ner_reddit_cooking-4.0.0-py3-none-any.whl models/en_ner_reddit_cooking-4.0.0-py3-none-any.whl
!cp -n packages/en_ner_reddit_cooking-5.0.0/dist/en_ner_reddit_cooking-5.0.0-py3-none-any.whl models/en_ner_reddit_cooking-5.0.0-py3-none-any.whl
!cp -n packages/en_ner_reddit_cooking-6.0.0/dist/en_ner_reddit_cooking-6.0.0-py3-none-any.whl models/en_ner_reddit_cooking-6.0.0-py3-none-any.whl

This now means that you can install any of the models using `pip install` like:

In [41]:
!python -m pip install models/en_ner_reddit_cooking-3.0.0-py3-none-any.whl

Processing ./models/en_ner_reddit_cooking-3.0.0-py3-none-any.whl

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


Then you can use in Python to load that model like:

In [42]:
import spacy
nlp = spacy.load("en_ner_reddit_cooking")
doc = nlp("Make a dressing with mayo, plain yogurt, curry powder, mustard powder, mustard, ground coriander.")
doc.ents

(mayo, plain yogurt, curry powder, mustard powder, mustard, ground coriander)

### Save models to HF Hub

You can then save your model to Hugging Face Hub too. For this, you'll need to have installed `spacy-huggingface_hub`, which is in the `requirements.txt`. You'll also need a HF Hub/Spaces account and set up your API token (see [link](https://huggingface.co/settings/tokens)).

In [ ]:
#!huggingface-cli login # recommend doing in terminal, not notebook

I want to only push my v3 model as it's lightweight (6MB), it's fast and still has good performance.

In [45]:
!python -m spacy huggingface-hub push models/en_ner_reddit_cooking-3.0.0-py3-none-any.whl

ℹ Publishing to repository 'wesslen/en_ner_reddit_cooking'
result /var/folders/bc/gqn95mfn5t1dmm0t3bp_t9nr0000gn/T/tmp_q9xxbyg/en_ner_reddit_cooking/meta.json en_ner_reddit_cooking/meta.json
result /var/folders/bc/gqn95mfn5t1dmm0t3bp_t9nr0000gn/T/tmp_q9xxbyg/en_ner_reddit_cooking/en_ner_reddit_cooking-3.0.0/README.md en_ner_reddit_cooking/en_ner_reddit_cooking-3.0.0/README.md
result /var/folders/bc/gqn95mfn5t1dmm0t3bp_t9nr0000gn/T/tmp_q9xxbyg/en_ner_reddit_cooking/en_ner_reddit_cooking-3.0.0/config.cfg en_ner_reddit_cooking/en_ner_reddit_cooking-3.0.0/config.cfg
result /var/folders/bc/gqn95mfn5t1dmm0t3bp_t9nr0000gn/T/tmp_q9xxbyg/en_ner_reddit_cooking/en_ner_reddit_cooking-3.0.0/meta.json en_ner_reddit_cooking/en_ner_reddit_cooking-3.0.0/meta.json
result /var/folders/bc/gqn95mfn5t1dmm0t3bp_t9nr0000gn/T/tmp_q9xxbyg/en_ner_reddit_cooking/en_ner_reddit_cooking-3.0.0/tokenizer en_ner_reddit_cooking/en_ner_reddit_cooking-3.0.0/tokenizer
result /var/folders/bc/gqn95mfn5t1dmm0t3bp_t9nr0000gn/T

If it runs successfully, you should get a final message like:
```
View your model here:https://huggingface.co/wesslen/en_ner_reddit_cooking

Install your model:
pip install https://huggingface.co/wesslen/en_ner_reddit_cooking/resolve/main/en_ner_reddit_cooking-any-py3-none-any.whl
```

You can view and test my model here: https://huggingface.co/wesslen/en_ner_reddit_cooking

### Appendix: Drop files

This is commented out and can be used to "clean" any Prodigy datasets and remove any models/packages.

In [43]:
# !python -m prodigy drop hmwk_1_zeroshot
# !python -m prodigy drop hmwk_1_workshop
# !python -m prodigy drop hmwk_1_eval
# !python -m prodigy drop hmwk_1_train_exp3
# !python -m prodigy drop hmwk_1_train_exp4
# !python -m prodigy drop hmwk_1_train_exp5
# !rm -rf output
# !rm -rf packages